### Create PNG slices from TIFF for each MECE mask

In [229]:
import os, sys, subprocess, psycopg2
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from PIL import Image, ImageFilter
from subprocess import call
from natsort import natsorted
from skimage import io, transform
import warnings; warnings.filterwarnings("ignore")

In [27]:
all_regions_tif = "./ZBrain2_0.tif"
all_regions = io.imread(all_regions_tif)

#### Obtain a stack for each region

In [50]:
regions_dict = dict()
n_regions = all_regions.max() + 1
for i in range(1, n_regions):
    regions_dict[i] = np.ndarray(all_regions.shape).astype(int)

In [74]:
for index, val in np.ndenumerate(all_regions):
    if val:
        regions_dict[val][index[0]][index[1]][index[2]] = 100

In [127]:
Image.fromarray(regions_dict[3][30]).show()

#### Import names for regions from LUT

In [145]:
df = pd.read_excel("./ZBrain2_0_LUT.xls", index_col=None, header=None)
df

,0,1,2,3
0,Forebrain,Diencephalon,Caudal Hypothalamus,NaN
1,Forebrain,Diencephalon,Habenula,NaN
2,Forebrain,Diencephalon,Intermediate Hypothalamus,NaN
3,Forebrain,Diencephalon,Intermediate Hypothalamus,Diffuse Nucleus of the Intermediate Hypothalamus
4,Forebrain,Diencephalon,Optic Tract and Accessory Optic System,NaN
5,Forebrain,Diencephalon,Optic Tract and Accessory Optic System,"Retinal Arborization Field 1 (AF1, Approximate..."
6,Forebrain,Diencephalon,Optic Tract and Accessory Optic System,"Retinal Arborization Field 2 (AF2, Approximate..."
7,Forebrain,Diencephalon,Optic Tract and Accessory Optic System,Retinal Arborization Field 3 (AF3)
8,Forebrain,Diencephalon,Optic Tract and Accessory Optic System,Retinal Arborization Field 4 (AF4)
9,Forebrain,Diencephalon,Optic Tract and Accessory Optic System,Retinal Arborization Field 5 (AF5)


In [175]:
lut = dict()

In [176]:
for i, row in df.iterrows():
    region_name = [c for c in row if isinstance(c, str)]
    region_name = ' - '.join(region_name)
    region_index = i + 1
    lut[region_index] = region_name

#### Create grayscale PNG slices for each region

In [221]:
regions_dir_path = "../app/assets/images/2-Regions/"

In [222]:
for i in range(1, n_regions):
    region_data, region_name = regions_dict[i].astype(int), lut[i]
    region_path = os.path.join(regions_dir_path, region_name)
    
    try:
        os.mkdir(region_path)
        os.mkdir(os.path.join(region_path, 'grayscale'))
    except:
        print('directory exists')
    finally:
        region_data = [np.rot90(im) for im in region_data]

        
    for i, im in enumerate(region_data):
        io.imsave("{0}/grayscale/{1}-{2}.png".format(region_path, region_name, i), im)

directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exists
directory exis

#### Convert PNG slices to alpha masks

In [223]:
regions_dir_path = "../app/assets/images/2-Regions/"

In [224]:
def convert_slices_to_outlines(input_pngs_dir):
    stack_dirs = natsorted([d for d in os.listdir(input_pngs_dir) if '.DS_Store' not in d])
    for stack in stack_dirs:
        stack_dir_path = os.path.join(input_pngs_dir, stack, 'grayscale')
        slices = [s for s in os.listdir(stack_dir_path) if '.DS_Store' not in s]
        for sli in slices:
            sli_path = os.path.join(stack_dir_path, sli)
            sli_image = Image.open(sli_path).convert("L")
            sli_image = sli_image.filter(ImageFilter.FIND_EDGES)
            sli_image.save(sli_path)

In [225]:
convert_slices_to_outlines(regions_dir_path)

#### Use Imagemagick to convert grayscale slices to colored

In [232]:
def convert_slices_to_alpha_masks(stack_dir_path, colors):
    for color in colors:
        mask_dir_path = os.path.join(stack_dir_path, color)
        try:
            os.mkdir(mask_dir_path)
        except:
            pass
        color_val = color
        if color is 'green':
            color_val = 'green1'
        try:
            subprocess.check_output(["mogrify", "-path", "{0}".format(mask_dir_path), "+level-colors", 
                                     "black,{0}".format(color_val), "-transparent", "black", 
                                     "{0}/grayscale/*.png".format(stack_dir_path)], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print(err.output)

In [233]:
colors = ['cyan', 'purple', 'green', 'yellow']
stack_dirs = sorted([d for d in os.listdir(regions_dir_path) if '.DS_Store' not in d])

for stack in stack_dirs:
    stack_dir_path = os.path.join(regions_dir_path, stack,)
    print(stack_dir_path)
    convert_slices_to_alpha_masks(stack_dir_path, colors)

../app/assets/images/2-Regions/Forebrain - Diencephalon - Caudal Hypothalamus
../app/assets/images/2-Regions/Forebrain - Diencephalon - Habenula
../app/assets/images/2-Regions/Forebrain - Diencephalon - Intermediate Hypothalamus
../app/assets/images/2-Regions/Forebrain - Diencephalon - Intermediate Hypothalamus - Diffuse Nucleus of the Intermediate Hypothalamus
../app/assets/images/2-Regions/Forebrain - Diencephalon - Optic Tract and Accessory Optic System
../app/assets/images/2-Regions/Forebrain - Diencephalon - Optic Tract and Accessory Optic System - Retinal Arborization Field 1 (AF1, Approximate Location)
../app/assets/images/2-Regions/Forebrain - Diencephalon - Optic Tract and Accessory Optic System - Retinal Arborization Field 2 (AF2, Approximate Location)
../app/assets/images/2-Regions/Forebrain - Diencephalon - Optic Tract and Accessory Optic System - Retinal Arborization Field 3 (AF3)
../app/assets/images/2-Regions/Forebrain - Diencephalon - Optic Tract and Accessory Optic Sys

In [94]:
tiffs_dir_path = "./RegionTiffs/"
tiff_stacks = natsorted(os.listdir(tiffs_dir_path))
tiff_stacks = [tiff.replace('.tif', '') for tiff in tiff_stacks]
tiff_stacks

FileNotFoundError: [Errno 2] No such file or directory: './RegionTiffs/'

In [3]:
mece_dir_path = "../app/assets/images/8-MeceMasks/"

#### Output them as PNG slices

In [5]:
def slice_tiff_stacks(tiff_stacks, input_tiffs_dir, output_pngs_dir):
    for tiff in tiff_stacks:
        input_tiff = input_tiffs_dir + tiff + '.tif'
        output_pngs = output_pngs_dir + tiff
        
        try:
            os.mkdir(output_pngs)
        except:
            print('directory exists')
        finally:
            stack = io.imread(input_tiff)
            stack = [np.rot90(im) for im in stack]
            
        for i, image in enumerate(stack):
            io.imsave("{0}/{1}-{2}.png".format(output_pngs, tiff, i), image)

In [33]:
slice_tiff_stacks(tiff_stacks, tiffs_dir_path, mece_dir_path)

#### Use PIL to trim pngs to outline pngs

In [34]:
mece_dir_path = "../app/assets/images/8-MeceMasks/"

In [35]:
def convert_slices_to_outlines(input_pngs_dir):
    stack_dirs = [d for d in os.listdir(input_pngs_dir) if '.DS_Store' not in d]
    for stack in stack_dirs:
        stack_dir_path = os.path.join(input_pngs_dir, stack)
        slices = [s for s in os.listdir(stack_dir_path) if '.DS_Store' not in s]
        for sli in slices:
            sli_path = os.path.join(stack_dir_path, sli)
            sli_image = Image.open(sli_path).convert("L")
            sli_image = sli_image.filter(ImageFilter.FIND_EDGES)
            sli_image.save(sli_path)
            

In [36]:
convert_slices_to_outlines(mece_dir_path)

#### Use ImageMagick to convert png to colored masks with alpha

In [37]:
def convert_tifpng_to_alpha_masks(stack_dir_path, colors):
    for color in colors:
        mask_dir_path = os.path.join(stack_dir_path, color)
        try:
            os.mkdir(mask_dir_path)
        except:
            pass
        color_val = color
        if color is 'green':
            color_val = 'green1'
        try:
            subprocess.check_output(["mogrify", "-path", "{0}".format(mask_dir_path), "+level-colors", "black,{0}".format(color_val), "-transparent", "black", "{0}/*.png".format(stack_dir_path)], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print(err.output)

In [ ]:
colors = ['cyan', 'purple', 'green', 'yellow']
stack_dirs = sorted([d for d in os.listdir(mece_dir_path) if '.DS_Store' not in d])

for stack in stack_dirs:
    stack_dir_path = os.path.join(mece_dir_path, stack)
    print(stack_dir_path)
    convert_tifpng_to_alpha_masks(stack_dir_path, colors)

../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Anterior group of the posterior tubercular vmat2 neurons
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Anterior preoptic dopaminergic cluster
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Anterior pretectum cluster of vmat2 Neurons
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Dopaminergic Cluster 1 - ventral thalamic and periventricular posterior tubercular DA neurons
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Dopaminergic Cluster 2 - posterior tuberculum
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Dopaminergic Cluster 3 - hypothalamus
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Dopaminergic Cluster 4-5 - posterior tuberculum and hypothalamus
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Dopaminergic Cluster 6 - hypothalamus
../app/assets/images/8-MeceMasks/Forebrain - Diencephalon - Dopaminergic Cluster 7 - Caudal Hypotha

### Prepare postgreSQL table

In [266]:
create_table_command = (
"""
    DROP TABLE IF EXISTS regions;
    CREATE TABLE regions (
        region_img_id SERIAL PRIMARY KEY,
        region_name TEXT NOT NULL,
        region_id INT NOT NULL,
        region_image_path TEXT NOT NULL,
        region_image_size INT,
        region_is_mece BOOLEAN NOT NULL
    )
""")

#### Connect to zbrain2db database and create table

In [267]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

#### Prepare table entries for regions

In [268]:
mece_regions_dir_path = "../app/assets/images/2-Regions/Mece/"
mece_region_dirs = [d for d in os.listdir(mece_regions_dir_path) if '.DS_Store' not in d]
mece_region_dirs = [os.path.join(mece_regions_dir_path, d) for d in mece_region_dirs]

In [269]:
slice_id = 2000
insert_commands = list()
for i, region_dir in enumerate(mece_region_dirs):
    region_dir = os.path.join(region_dir, 'grayscale')
    slices = natsorted([d for d in os.listdir(region_dir) if '.DS_Store' not in d])
    region_name = region_dir.split('/')[-2]
    region_is_mece = True
    for j, slice_path in enumerate(slices):
        slice_size = os.stat(os.path.join(region_dir, slice_path)).st_size
    
        insert_command = ("INSERT INTO regions (region_img_id, region_name, region_id, region_image_path, region_image_size, region_is_mece) values ({0}, '{1}', {2}, '{3}', {4}, {5})"
                    .format(slice_id, region_name.replace("'", "''"), i, slice_path.replace("'", "''"), slice_size, region_is_mece))
        slice_id += 1
        insert_commands.append(insert_command)

In [270]:
print(slice_id, len(insert_commands), insert_commands[0])

18422 16422 INSERT INTO regions (region_img_id, region_name, region_id, region_image_path, region_image_size, region_is_mece) values (2000, 'Forebrain - Diencephalon - Rostral Hypothalamus', 0, 'Forebrain - Diencephalon - Rostral Hypothalamus-0.png', 927, True)


In [271]:
other_regions_dir_path = "../app/assets/images/2-Regions/Other/"
other_region_dirs = [d for d in os.listdir(other_regions_dir_path) if '.DS_Store' not in d]
other_region_dirs = [os.path.join(other_regions_dir_path, d) for d in other_region_dirs]

In [272]:
for i, region_dir in enumerate(other_region_dirs):
    region_dir = os.path.join(region_dir, 'grayscale')
    slices = natsorted([d for d in os.listdir(region_dir) if '.DS_Store' not in d])
    region_name = region_dir.split('/')[-2]
    region_is_mece = False
    for j, slice_path in enumerate(slices):
        slice_size = os.stat(os.path.join(region_dir, slice_path)).st_size
    
        insert_command = ("INSERT INTO regions (region_img_id, region_name, region_id, region_image_path, region_image_size, region_is_mece) values ({0}, '{1}', {2}, '{3}', {4}, {5})"
                    .format(slice_id, region_name.replace("'", "''"), i, slice_path.replace("'", "''"), slice_size, region_is_mece))
        slice_id += 1
        insert_commands.append(insert_command)

In [273]:
print(slice_id, len(insert_commands), insert_commands[-1])

44504 42504 INSERT INTO regions (region_img_id, region_name, region_id, region_image_path, region_image_size, region_is_mece) values (44503, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-137.png', 927, False)


In [274]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception as e:
        print(e)
        
cursor.close()
conn.commit()

#### Query database

In [277]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM regions ORDER BY region_img_id')

rows = cursor.fetchall()
for row in rows[-100:-1]:
    print(row)
cursor.close()
conn.close()

(44404, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-38.png', 927, False)
(44405, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-39.png', 927, False)
(44406, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-40.png', 927, False)
(44407, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-41.png', 927, False)
(44408, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-42.png', 927, False)
(44409, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-43.png', 927, False)
(44410, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-44.png', 927, False)
(44411, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-45.png', 927, False)
(44412, 'Hindbrain - Cerebellar Neuropil 1', 188, 'Hindbrain - Cerebellar Neuropil 1-46.png', 927, False)
(44413, 'Hindbrain - Cerebellar Neuropil 1', 1